In [7]:
import numpy as np
import healpy as hp
import math as mh
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [1]:
L = 2048
int(L * (L + 1) / 2)

2098176

In [12]:
CMB_map_path = '../data/planck_simulation/ffp10_newdust_total_030_full_map.fits'
original_healpix_map = hp.read_map(CMB_map_path)

In [17]:
hp.get_nside(original_healpix_map)*2

2048

In [18]:
alm = hp.map2alm(original_healpix_map, lmax=hp.get_nside(original_healpix_map)*2)

In [19]:
L_max = hp.get_nside(original_healpix_map)*2

In [20]:
# Step 3: Rearrange coefficients for s2wav: from 1 dimensional to 2 dimensional
# s2fft only works with alm in 2d
alm_2d = np.zeros((L_max, 2 * L_max - 1), dtype=np.complex128)

for l in range(L_max):
    for m in range(-l, l + 1):
        index = hp.Alm.getidx(L_max - 1, l, abs(m))
        if m < 0:
            alm_2d[l, L_max + m - 1] = (-1)**m * np.conj(alm[index])
        else:
            alm_2d[l, L_max + m - 1] = alm[index]
print(f"Shape of alm_2d: {alm_2d.shape}")

Shape of alm_2d: (2048, 4095)


In [15]:
import numpy as np
import healpy as hp

def rearrange_alm(hp_alm, lmax, Store=False, path=None):
    # Initialize the 2D MW_alm array with correct dimensions
    MW_alm = np.zeros((lmax, 2 * lmax + 1), dtype=np.complex128)

    for l in range(lmax):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax - 1, l, abs(m))
            if m < 0:
                MW_alm[l, lmax + m] = (-1)**m * np.conj(hp_alm[index])
            else:
                MW_alm[l, lmax + m] = hp_alm[index]

    # Optionally store the array
    if Store:
        if path is None:
            np.save("MW_alm.npy", MW_alm)
        else:
            np.save(path, MW_alm)
    
    return MW_alm

# Example usage
lmax = 4
# Note: In a real scenario, hp_alm would be generated or provided by some spherical harmonic transform
# hp_alm = np.random.randn((lmax * (lmax + 1)) // 2) + 1j * np.random.randn((lmax * (lmax + 1)) // 2)
hp_alm = hp.map2alm(original_healpix_map)
MW_alm = rearrange_alm(hp_alm, hp.get_nside(original_healpix_map)*2)

print(MW_alm.shape)

(2048, 4097)


In [21]:
s2fft.forward(alm_2d,alm_2d.shape[0])

KeyboardInterrupt: 

In [22]:
s2fft.forward(MW_alm,MW_alm.shape[0])

KeyboardInterrupt: 

In [ ]:
def zeropad(i): #(alms[HPX],scale_lmax,smoothing_lmax) - COULD RE-WRITE FOR MW ALM'S???
    print("Zero-padding the alm's")

    #Pre-allocate array for enlarged alm's
    new_alms = np.zeros(((i[2]*(i[2]+1))/2.),dtype=complex)
    for em in range(i[1]):
        startindex = em*i[1] - .5*em*(em-1)
        new_startindex = em*i[2] - .5*em*(em-1)
        new_alms[new_startindex:(new_startindex+i[1]-em)] = i[0][startindex:(startindex+i[1]-em)]

    return new_alms


In [ ]:
def zeropad(i):
    print("Zero-padding the alm's")

    #Pre-allocate array for enlarged alm's
    new_alms = np.zeros(((i[2]*(i[2]+1))/2.),dtype=complex)
    for em in range(i[1]):
        startindex = em*i[1] - .5*em*(em-1)
        new_startindex = em*i[2] - .5*em*(em-1)
        new_alms[new_startindex:(new_startindex+i[1]-em)] = i[0][startindex:(startindex+i[1]-em)]

    return new_alms

In [ ]:
# Step 1 Scale up all the wavelet coefficient maps to 2*l by adding zero paddings

#Doubling lmax for input maps with zero-padding
#Serial version
'''mapsdouble = np.zeros((nrows,ps.mw_size(smoothing_lmax)),dtype=np.complex128) #Pre-allocate array
for i in xrange(nrows):
    mapsdouble[i,:] = doubleworker((mapsextra[0][i],mapsextra[1],smoothing_lmax,mapsextra[2]))'''
#Parallel version
mapsextra2 = [(mapsextra[0],mapsextra[1],i,scale_lmax,smoothing_lmax) for i in xrange(nmaps)]
print "Forming pool"
pool2 = mg.Pool(nprocess2)
print "Farming out workers to run doubling function"
double_output = pool2.map(doubleworker,mapsextra2)
print "Have returned from doubling workers\n"
pool2.close()
pool2.join()
del pool2


